In [1]:

import math

def theory_to_elas(beta, tau, H0, C_delta=1.0, C_Omega=1.0, phi0=0.0):
    """
    Entrée  : beta(>0), tau(>0), H0 (>0), C_delta≈1, C_Omega≈1
    Sortie  : (delta, Omega, phi) pour l’ansatz ELAS
    Formules: delta ≈ C_delta * beta * H0**2
              Omega ≈ C_Omega / (H0 * tau)
              phi = phi0 (ou ajustée par fit)
    """
    assert beta > 0 and tau > 0 and H0 > 0
    delta = C_delta * beta * H0**2
    Omega = C_Omega / (H0 * tau)
    phi = phi0
    return delta, Omega, phi

def elas_to_theory(delta, Omega, H0, C_delta=1.0, C_Omega=1.0):
    """
    Entrée  : delta(≥0), Omega(>0), H0 (>0), C_delta≈1, C_Omega≈1
    Sortie  : (beta, tau) en normalisation de l’Elastic Gravity
    Formules: beta ≈ delta / (C_delta * H0**2)
              tau  ≈ C_Omega / (H0 * Omega)
    """
    assert delta >= 0 and Omega > 0 and H0 > 0
    beta = delta / (C_delta * H0**2)
    tau  = C_Omega / (H0 * Omega)
    return beta, tau

def alpha_from_beta_tau(beta, tau):
    """
    Rappels: beta = alpha * tau**2  ->  alpha = beta / tau**2
    """
    assert beta > 0 and tau > 0
    return beta / (tau**2)

In [2]:

def sigma_beta_from_sigma_delta(sigma_delta, H0, C_delta=1.0):
    return sigma_delta / (C_delta * H0**2)

def sigma_tau_from_sigma_Omega(Omega, sigma_Omega, H0, C_Omega=1.0):
    return (C_Omega / H0) * (sigma_Omega / (Omega**2))

def sigma_alpha_from(sig_beta, beta, sig_tau, tau, corr_beta_tau=0.0):
    """
    (σ_α/α)^2 ≈ (σ_β/β)^2 + 4(σ_τ/τ)^2 - 4ρ (σ_β/β)(σ_τ/τ)
    """
    rel2 = (sig_beta/beta)**2 + 4*(sig_tau/tau)**2 - 4*corr_beta_tau*(sig_beta/beta)*(sig_tau/tau)
    return (beta/(tau**2)) * math.sqrt(max(rel2, 0.0))

In [3]:

import numpy as np
from math import log
# from scipy.optimize import curve_fit  # si tu l’as

def elas_ansatz(z, delta, Omega, phi, rho0=1.0):
    return rho0 * (1.0 + delta * np.cos(Omega * np.log(1.0 + z) + phi))

def calibrate_Cdelta_COmega(beta, tau, H0, z_grid, rhoLambda_from_model):
    """
    beta, tau, H0 : paramètres théorie
    z_grid        : array de z (ex. np.linspace(0, 2, 200))
    rhoLambda_from_model(z) : fonction qui renvoie ρ_Λ(z) calculée à partir de ta théorie (T3)
                              (à défaut, approx via E(z) et séparation composantes)
    Retour : (C_delta, C_Omega, fit_params)
    """
    rho = rhoLambda_from_model(z_grid)              # shape (N,)
    rho0 = rho[0] if z_grid[0] == 0.0 else np.mean(rho[z_grid < 0.05])
    y = rho / rho0 - 1.0                            # normalisé (≈ delta cos(...))

    # Fit simple sans SciPy (grid-search grossier) — robuste si tu n’as pas curve_fit :
    deltas = np.linspace(0.01, 0.30, 60)
    Omegas = np.linspace(0.10, 3.00, 120)
    phis   = np.linspace(-np.pi, np.pi, 90)
    best = (1e99, None)
    for d in deltas:
        for Om in Omegas:
            # petit truc: un fit linéaire en cos & sin pour estimer phi rapidement
            Xc = np.cos(Om*np.log(1+z_grid))
            Xs = np.sin(Om*np.log(1+z_grid))
            # ajuste a,b dans y ≈ d*(a*Xc + b*Xs) => phi = atan2(b,a)
            A = np.vstack([Xc, Xs]).T
            coef, *_ = np.linalg.lstsq(A, y, rcond=None)
            a, b = coef
            phi = np.arctan2(b, a)
            y_th = d*np.cos(Om*np.log(1+z_grid) + phi)
            err = np.mean((y - y_th)**2)
            if err < best[0]:
                best = (err, (d, Om, phi))

    delta_num, Omega_num, phi_num = best[1]

    # Calibrations
    C_delta = (delta_num) / (beta * H0**2)
    C_Omega = (Omega_num) * (H0 * tau)

    fit_params = {"delta": delta_num, "Omega": Omega_num, "phi": phi_num, "rho0": rho0}
    return C_delta, C_Omega, fit_params